Data misfit in time-domain.

#### Notebook config

In [10]:
%autoreload 2

In [14]:
import fwipy 
print(fwipy.__file__)
from fwipy import *
from fwipy.config.logging import *
lll(40)
from fwipy.ioapi.segy import SgyFile


/home/kajetan/software/Kolumbo-FWI/fwipy/__init__.py


#### Notebook config